In [1]:
!nvidia-smi

Fri Jun  3 01:36:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 28.3 MB/s 
     |████████████████████████████████| 1.2 MB 55.2 MB/s 
     |████████████████████████████████| 6.6 MB 44.2 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 70.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoConfig,get_linear_schedule_with_warmup,RobertaModel,RobertaPreTrainedModel
from torch.utils.data import DataLoader
import torch
from create_dataloaders import create_enssemble_dataloader
import numpy as np
from os.path import join
from tqdm.auto import tqdm
import os
from os.path import join
from torch import nn
from sklearn.metrics import *
from torch.nn import CrossEntropyLoss
from transformers.modeling_outputs import SequenceClassifierOutput

In [2]:
pre_dataset_directory="/content/drive/MyDrive/en-vi-nli/pre_dataset"

In [3]:
train=pd.read_csv(f'{pre_dataset_directory}/train/train_60_100k.csv')
valid=pd.read_csv(f'{pre_dataset_directory}/valid/valid.csv')

In [4]:
names=['xlm-roberta-base','bert-base-multilingual-cased']

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
tokenizers=[AutoTokenizer.from_pretrained(name) for name in names]

In [8]:
train_batch_size=16
train_premises=train['norm_premise'].tolist()
train_hypothesises=train['norm_hypothesis'].tolist()
train_labels=train['label'].tolist()

valid_batch_size=16
valid_premises=valid['norm_premise'].tolist()
valid_hypothesises=valid['norm_hypothesis'].tolist()
valid_labels=valid['label'].tolist()

In [9]:
max_length=60

In [10]:
train_dataloader=create_enssemble_dataloader(tokenizers,train_premises,train_hypothesises,train_labels,max_length,train_batch_size,device=device,shuffle=False)

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/100000 [00:00<?, ?it/s]

In [11]:
valid_dataloader=create_enssemble_dataloader(tokenizers,valid_premises,valid_hypothesises,valid_labels,max_length,valid_batch_size,device=device,shuffle=False)

  0%|          | 0/4980 [00:00<?, ?it/s]

  0%|          | 0/4980 [00:00<?, ?it/s]

In [12]:
class Classifier(nn.Module):
  def __init__(self, dense_dim_in,dense_dim_out,num_labels,drop_rate):
    super(Classifier,self).__init__()

    self.dense_dim_in=dense_dim_in
    self.dense_dim_out=dense_dim_out
    self.drop_rate=drop_rate
    self.num_labels=num_labels

    self.dense = nn.Linear(self.dense_dim_in, self.dense_dim_out)
    
    self.dropout = nn.Dropout(self.drop_rate)
    self.out_proj = nn.Linear(self.dense_dim_out, self.num_labels)
  
  def forward(self,inputs):
    #inputs: (batch_size,dense_dim_in)
    inputs=self.dense(inputs)
    inputs = self.dense(inputs)
    inputs = torch.tanh(inputs)
    inputs = self.dropout(inputs)
    logits = self.out_proj(inputs)

    return logits

In [53]:
labels=[0,1,2]
num_labels=len(labels)
classifier_dropout=0.1
config=AutoConfig.from_pretrained('bert-base-multilingual-cased')
config.classifier_dropout=classifier_dropout
config.num_labels=num_labels
config_directory='/content/drive/MyDrive/en-vi-nli/models/bert-base-multilingual-cased'
os.makedirs(config_directory,exist_ok=True)
config.save_pretrained(config_directory)

#Init weights classifier for rembert

In [23]:
names[1]

'bert-base-multilingual-cased'

In [24]:
labels=[0,1,2]
num_labels=len(labels)
classifier_dropout=0.1
config=AutoConfig.from_pretrained(names[1])
config.classifier_dropout=classifier_dropout
config.num_labels=num_labels


In [25]:
model_directory='/content/drive/MyDrive/en-vi-nli/models/last_layer_classifier'
os.makedirs(model_directory,exist_ok=True)

In [26]:
four_last_layer_classifier=Classifier(config.hidden_size,config.hidden_size,num_labels,config.classifier_dropout)
torch.save({'last_layer_classifier':four_last_layer_classifier.state_dict()},
           join(model_directory,'last_layer_classifier2.pt'))

#last layer xlm-roberta-base

In [13]:
class XLMRoBERTa4LastLayersForClassification(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.config = config
        
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        
        self.classifier=self.load_weight_classifier()

        self.init_weights()

    def load_weight_classifier(self):
        classifier_ckpt=torch.load('/content/drive/MyDrive/en-vi-nli/models/last_layer_classifier/last_layer_classifier.pt')
        classifier=Classifier(self.config.hidden_size,self.config.hidden_size,self.config.num_labels,self.config.classifier_dropout)
        classifier.load_state_dict(classifier_ckpt['last_layer_classifier'])
        return classifier
    
    def forward(self,input_ids,attention_mask,labels=None):

        outputs = self.roberta(input_ids=input_ids,attention_mask=attention_mask,output_hidden_states=False)
        # last_layers=torch.stack(outputs.hidden_states[-4:])
        # last_layers=last_layers.permute(1,2,0,3)
        # features=torch.flatten(last_layers,start_dim=2)
        # cls_embedding=features[:,0,:]
        cls_embedding=outputs[0][:,0,:]
        
        logits=self.classifier(cls_embedding)

        loss=0
        if labels!=None:
          loss_fct = CrossEntropyLoss()
          loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
                loss=loss,
                logits=logits)

#last layer mbert-base-cased

In [14]:
from transformers.models.rembert.modeling_rembert import RemBertModel,RemBertPreTrainedModel

In [15]:
from transformers import BertPreTrainedModel,BertModel

In [16]:
class mBert4LastLayersForClassification(BertPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.config = config
        
        self.bert = BertModel(config, add_pooling_layer=False)
        
        self.classifier=self.load_weight_classifier()

        self.init_weights()

    def load_weight_classifier(self):
        classifier_ckpt=torch.load('/content/drive/MyDrive/en-vi-nli/models/last_layer_classifier/last_layer_classifier2.pt')
        classifier=Classifier(self.config.hidden_size,self.config.hidden_size,self.config.num_labels,self.config.classifier_dropout)
        classifier.load_state_dict(classifier_ckpt['last_layer_classifier'])
        return classifier
    
    def forward(self,input_ids,attention_mask,token_type_ids,labels=None):

        outputs = self.bert(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,output_hidden_states=False)
        # last_layers=torch.stack(outputs.hidden_states[-4:])
        # last_layers=last_layers.permute(1,2,0,3)
        # features=torch.flatten(last_layers,start_dim=2)
        # cls_embedding=features[:,0,:]
        cls_embedding=outputs[0][:,0,:]
        
        logits=self.classifier(cls_embedding)

        loss=0
        if labels!=None:
          loss_fct = CrossEntropyLoss()
          loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return SequenceClassifierOutput(
                loss=loss,
                logits=logits)

In [17]:
class EnsembleModel(nn.Module):
  def __init__(self,config_1,config_2):
    super(EnsembleModel, self).__init__()
    self.config_1=config_1
    self.config_2=config_2
    self.name_1=self.config_1._name_or_path
    self.name_2=self.config_2._name_or_path
    self.model_1=XLMRoBERTa4LastLayersForClassification.from_pretrained(self.name_1,config=self.config_1)#.to(device)
    self.model_2=mBert4LastLayersForClassification.from_pretrained(self.name_2,config=self.config_2)#.to(device)

  def forward(self,input_ids_1,attention_mask_1,input_ids_2,attention_mask_2,token_type_ids_2,labels=None):
    outputs_1=self.model_1(input_ids=input_ids_1,attention_mask=attention_mask_1,labels=labels)
    outputs_2=self.model_2(input_ids=input_ids_2,attention_mask=attention_mask_2,token_type_ids=token_type_ids_2,labels=labels)

    loss=0
    if labels!=None:
      loss=(outputs_1.loss+outputs_2.loss)/2
    logits=(outputs_1.logits+outputs_2.logits)/2

    return SequenceClassifierOutput(
                loss=loss,
                logits=logits)



In [18]:
classifier_dropout=0.1
num_labels=3

In [19]:
config_1=AutoConfig.from_pretrained('/content/drive/MyDrive/en-vi-nli/models/xlm-roberta-base/config.json')
config_2=AutoConfig.from_pretrained('/content/drive/MyDrive/en-vi-nli/models/bert-base-multilingual-cased/config.json')
config_1._name_or_path='xlm-roberta-base'
config_2._name_or_path='bert-base-multilingual-cased'
config_1.classifier_dropout=classifier_dropout
config_1.num_labels=num_labels
config_2.classifier_dropout=classifier_dropout
config_2.num_labels=num_labels

In [21]:
model=EnsembleModel(config_1,config_2).to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRoBERTa4LastLayersForClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRoBERTa4LastLayersForClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRoBERTa4LastLayersForClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRoBERTa4LastLayersForClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['clas

In [22]:
import gc
gc.collect()

331

In [23]:
epochs=30
init_lr=1e-5
eps =1e-8

In [24]:
optimizer=torch.optim.AdamW(model.parameters(),lr=init_lr,weight_decay =0.01,eps=eps)

In [25]:
def save_model(epoch,model,optimizer,scheduler,training_loss,valid_loss,train_acc,valid_acc,path,step=None):
  torch.save({
          'epoch': epoch,
          'model_state_dict': model,
          'optimizer_state_dict': optimizer,
          'scheduler_state_dict': scheduler,
          'training_loss': training_loss,
          'valid_loss': valid_loss,
          'train_acc': train_acc,
          'valid_acc': valid_acc,
          'step':step
          }, join(path,f'ckpt{epoch}.pt'))

In [26]:
def run_train(optimizer,dataloader,lr_scheduler=None):
  model.train(True)
  loop = tqdm(dataloader)
  loss=0
  true=[]
  pred=[]
  for batch in dataloader:
    optimizer.zero_grad()
    inputs=batch['inputs']
    inputs.update({"labels":batch['labels']})
    true+=batch['labels'].tolist()
    outputs=model(**inputs)
    outputs.loss.backward()

    loss+=outputs.loss.item()
    probs=torch.nn.functional.softmax(outputs.logits,dim=-1)
    _pred=torch.argmax(probs,dim=-1).tolist()
    pred+=_pred
    optimizer.step()
    if lr_scheduler!=None:
      lr_scheduler.step()
    loop.set_postfix({f'train_loss_per_batch':outputs.loss.item()})
    loop.update()      
  acc=accuracy_score(true,pred)
  return loss/len(dataloader),acc

In [27]:
def run_valid(dataloader):
  model.eval()
  loss=0
  pred=[]
  true=[]
  with torch.no_grad():
    for batch in dataloader:
      inputs=batch['inputs']
      inputs.update({"labels":batch['labels']})
      true+=batch['labels'].tolist()
      
      outputs=model(**inputs)
      probs=torch.nn.functional.softmax(outputs.logits,dim=-1)
      _pred=torch.argmax(probs,dim=-1).tolist()
      pred+=_pred
      loss+=outputs.loss.item()

  acc=accuracy_score(true,pred)

  return loss/len(dataloader),acc

In [28]:
import gc
gc.collect()

100

In [29]:
torch.cuda.empty_cache()

In [30]:
lr_scheduler=None

In [31]:
model_directory='/content/drive/MyDrive/en-vi-nli/models/last-layer-ensemble'
os.makedirs(model_directory,exist_ok=True)
model_directory

'/content/drive/MyDrive/en-vi-nli/models/last-layer-ensemble'

In [32]:
from GPUtil import showUtilization as gpu_usage

gpu_usage()

| ID | GPU | MEM |
------------------
|  0 |  0% | 21% |


In [ ]:
train_loss_per_step=[]
valid_loss_per_step=[]
train_acc_per_step=[]
valid_acc_per_step=[]
# min_loss=np.inf
best_acc=0
count_stopping=0
for epoch in range(epochs):

  train_loss,train_acc=run_train(optimizer,train_dataloader,lr_scheduler)
  
  valid_loss,valid_acc=run_valid(valid_dataloader)

  train_loss_per_step.append(train_loss)
  valid_loss_per_step.append(valid_loss)

  train_acc_per_step.append(train_acc)
  valid_acc_per_step.append(valid_acc)
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss} train_acc: {train_acc} valid_acc: {valid_acc}')
  # if valid_loss<min_loss:
  if best_acc<valid_acc:
    # min_loss=valid_loss
    best_acc=valid_acc
    save_model(epoch,model.state_dict(),optimizer.state_dict(),lr_scheduler,train_loss,valid_loss,train_acc,valid_acc,model_directory)
    count_stopping=0
  else:
    count_stopping+=1
  
  if count_stopping>5:
    break


  0%|          | 0/6250 [00:00<?, ?it/s]